In [1]:
import rasterio

In [10]:
import numpy as np

In [2]:
img_dir = 'N:/dataorg-datasets/MLsatellite/sentinel2_images/images_danya/'

In [6]:
with rasterio.open(img_dir + 'geotiff/L2A_T43SFR_A023697_20200105T053922.tiff') as src:
    meta_43sfr = src.meta
    
with rasterio.open(img_dir + '43RGQ/safe/S2A_MSIL1C_20200105T053221_N0208_R105_T43RGQ_20200105T072254.SAFE/GRANULE/L1C_T43RGQ_A023697_20200105T053922/IMG_DATA/T43RGQ_20200105T053221_B02.jp2') as src:
    meta_43rgq = src.meta

In [7]:
meta_43sfr

{'driver': 'JP2OpenJPEG',
 'dtype': 'uint16',
 'nodata': None,
 'width': 10980,
 'height': 10980,
 'count': 5,
 'crs': None,
 'transform': Affine(10.0, 0.0, 600000.0,
        0.0, -10.0, 3600000.0)}

In [15]:
band_sfr = np.ones((meta_43sfr['height'], meta_43sfr['width']))

In [8]:
meta_43rgq

{'driver': 'JP2OpenJPEG',
 'dtype': 'uint16',
 'nodata': None,
 'width': 10980,
 'height': 10980,
 'count': 1,
 'crs': None,
 'transform': Affine(10.0, 0.0, 699960.0,
        0.0, -10.0, 3500040.0)}

In [14]:
band_rgq = np.zeros((meta_43rgq['height'], meta_43rgq['width']))

### try 1: set src as None

=> become extremely small, but values are not changed, location is wrong.

In [79]:
with rasterio.open('../preds/from.tiff', 'r') as src:
    meta_from = src.meta
    band_from = src.read(1)        

In [80]:
np.unique(band_from)

array([2, 3], dtype=uint8)

In [81]:
meta_from

{'driver': 'GTiff',
 'dtype': 'uint8',
 'nodata': None,
 'width': 1892,
 'height': 2357,
 'count': 1,
 'crs': CRS.from_epsg(4326),
 'transform': Affine(9.810891648968448e-05, 0.0, 77.03211244126558,
        0.0, -9.810891648968448e-05, 32.25451163012633)}

In [82]:
meta_None = meta_from.copy()
meta_None.update(crs=None)
meta_None

{'driver': 'GTiff',
 'dtype': 'uint8',
 'nodata': None,
 'width': 1892,
 'height': 2357,
 'count': 1,
 'crs': None,
 'transform': Affine(9.810891648968448e-05, 0.0, 77.03211244126558,
        0.0, -9.810891648968448e-05, 32.25451163012633)}

In [83]:
with rasterio.open('../preds/to_None.tiff', 'w', **meta_None) as dst:
    dst.write_band(1, band)

with rasterio.open('../preds/to_None.tiff', 'r') as dst:
    to_None = dst.read(1)
    meta_to_None = dst.meta

In [84]:
np.unique(to_None)

array([2, 3], dtype=uint8)

### try 2: crs=None, transform=some unaffine one??

=> size seems not changed, values are ok not changed, location is wrong.

In [85]:
meta_None_transformed = meta_from.copy()
meta_None_transformed.update(count=2, crs=None, transform=meta_43sfr['transform'])
meta_None_transformed

{'driver': 'GTiff',
 'dtype': 'uint8',
 'nodata': None,
 'width': 1892,
 'height': 2357,
 'count': 2,
 'crs': None,
 'transform': Affine(10.0, 0.0, 600000.0,
        0.0, -10.0, 3600000.0)}

In [87]:
with rasterio.open('../preds/to_None_transformed.tiff', 'w', **meta_None_transformed) as dst:      
    dst.write_band(1, band_from)

In [88]:
with rasterio.open('../preds/to_None_transformed.tiff', 'r') as dst:
    to_None_transformed = dst.read(1)

In [89]:
np.unique(to_None_transformed)

array([2, 3], dtype=uint8)

### try 3

=> empty

In [90]:
meta_transformed = meta_from.copy()
meta_transformed.update(transform=meta_43rgq['transform'])
meta_transformed

{'driver': 'GTiff',
 'dtype': 'uint8',
 'nodata': None,
 'width': 1892,
 'height': 2357,
 'count': 1,
 'crs': CRS.from_epsg(4326),
 'transform': Affine(10.0, 0.0, 699960.0,
        0.0, -10.0, 3500040.0)}

In [91]:
with rasterio.open('../preds/to_transformed.tiff', 'w', **meta_transformed) as dst:    
    dst.write_band(1, band_from)
    
with rasterio.open('../preds/to_transformed.tiff', 'r') as dst:
    to_transformed = dst.read(1)

In [92]:
np.unique(to_transformed)

array([2, 3], dtype=uint8)

In [93]:
to_transformed

array([[3, 3, 3, ..., 3, 3, 3],
       [3, 3, 3, ..., 3, 3, 3],
       [3, 3, 3, ..., 3, 3, 3],
       ...,
       [3, 3, 3, ..., 3, 3, 3],
       [3, 3, 3, ..., 3, 3, 3],
       [3, 3, 3, ..., 3, 3, 3]], dtype=uint8)

### try 4

In [74]:
meta_43rgq

{'driver': 'JP2OpenJPEG',
 'dtype': 'uint16',
 'nodata': None,
 'width': 10980,
 'height': 10980,
 'count': 1,
 'crs': None,
 'transform': Affine(10.0, 0.0, 699960.0,
        0.0, -10.0, 3500040.0)}

In [75]:
with rasterio.open('../preds/to_43rgq.tiff', 'w', **meta_43rgq) as dst:     
    dst.write_band(1, band_from)

In [77]:
with rasterio.open('../preds/to_43rgq.tiff', 'r') as dst:
    to_43rgq = dst.read(1)

In [78]:
np.unique(to_43rgq)

array([1, 2, 3, 4], dtype=uint16)

### try 5

In [119]:
meta_43rgq_driver = meta_43rgq.copy()
meta_43rgq_driver.update(driver='GTiff')
meta_43rgq_driver

{'driver': 'GTiff',
 'dtype': 'uint16',
 'nodata': None,
 'width': 10980,
 'height': 10980,
 'count': 1,
 'crs': None,
 'transform': Affine(10.0, 0.0, 699960.0,
        0.0, -10.0, 3500040.0)}

In [120]:
with rasterio.open('../preds/to_43rgq_driver.tiff', 'w', **meta_43rgq_driver) as dst:     
    dst.write_band(1, band_from)

In [121]:
with rasterio.open('../preds/to_43rgq_driver.tiff', 'r') as dst:
    to_43rgq_driver = dst.read(1)

In [122]:
np.unique(to_43rgq_driver)

array([2, 3], dtype=uint16)

### others

In [35]:
from rasterio.transform import Affine
x = np.linspace(-4.0, 4.0, 240)
y = np.linspace(-3.0, 3.0, 180)
res = (x[-1] - x[0]) / 240.0
transform = Affine.translation(x[0] - res / 2, y[0] - res / 2) * Affine.scale(res, res)
transform

Affine(0.03333333333333333, 0.0, -4.016666666666667,
       0.0, 0.03333333333333333, -3.0166666666666666)

In [37]:
res

0.03333333333333333

### convert jp2 to tiff

In [108]:
with rasterio.open(img_dir + r'safe\S2A_MSIL1C_20200105T053221_N0208_R105_T43SFR_20200105T072254.SAFE\GRANULE\L1C_T43SFR_A023697_20200105T053922\IMG_DATA\T43SFR_20200105T053221_B02.jp2', 'r') as f:
    b02 = f.read(1)
    b02_meta = f.meta

In [109]:
b02.max(), b02.mean(), b02.min()

(15579, 3347.2681797754485, 0)

In [110]:
b02_meta

{'driver': 'JP2OpenJPEG',
 'dtype': 'uint16',
 'nodata': None,
 'width': 10980,
 'height': 10980,
 'count': 1,
 'crs': None,
 'transform': Affine(10.0, 0.0, 600000.0,
        0.0, -10.0, 3600000.0)}

In [111]:
b02_meta_to = b02_meta.copy()
b02_meta_to.update(driver='GTiff')
b02_meta_to

{'driver': 'GTiff',
 'dtype': 'uint16',
 'nodata': None,
 'width': 10980,
 'height': 10980,
 'count': 1,
 'crs': None,
 'transform': Affine(10.0, 0.0, 600000.0,
        0.0, -10.0, 3600000.0)}

In [112]:
with rasterio.open('../preds/b02.tiff', 'w', **b02_meta_to) as srs:
    srs.write_band(1, b02)

In [113]:
with rasterio.open('../preds/b02.tiff', 'r') as f:
    b02_to = f.read(1)

In [114]:
b02_to.max(), b02_to.mean(), b02_to.min()

(15579, 3347.2681797754485, 0)

In [115]:
b02_to

array([[   0,    0,    0, ..., 9218, 9394, 9496],
       [   0,    0,    0, ..., 8931, 9285, 9550],
       [   0,    0,    0, ..., 8526, 8996, 9357],
       ...,
       [   0,    0,    0, ..., 2122, 2122, 2202],
       [   0,    0,    0, ..., 2124, 2116, 2218],
       [   0,    0,    0, ..., 2119, 2072, 2080]], dtype=uint16)

In [116]:
b02

array([[   0,    0,    0, ..., 9218, 9394, 9496],
       [   0,    0,    0, ..., 8931, 9285, 9550],
       [   0,    0,    0, ..., 8526, 8996, 9357],
       ...,
       [   0,    0,    0, ..., 2122, 2122, 2202],
       [   0,    0,    0, ..., 2124, 2116, 2218],
       [   0,    0,    0, ..., 2119, 2072, 2080]], dtype=uint16)